## 数据初探

In [1]:
import pandas as pd

jet = pd.read_table('train_data.txt',sep=' ',header=None)
jet.drop([26,27],axis=1,inplace=True)

jet_index_1 = jet.iloc[:,0].tolist()
jet_index_2 = jet.iloc[:,1].tolist()

jet_index = pd.MultiIndex.from_arrays([jet_index_1,jet_index_2],names=['id','flight'])

jet.set_index(jet_index,inplace=True)
jet.drop([0,1],axis=1,inplace=True)
jet

2       3      4       5       6        7        8      9   \
id  flight                                                                    
1   1       34.9983  0.8400  100.0  449.44  555.32  1358.61  1137.23   5.48   
    2       41.9982  0.8408  100.0  445.00  549.90  1353.22  1125.78   3.91   
    3       24.9988  0.6218   60.0  462.54  537.31  1256.76  1047.45   7.05   
    4       42.0077  0.8416  100.0  445.00  549.51  1354.03  1126.38   3.91   
    5       25.0005  0.6203   60.0  462.54  537.07  1257.71  1047.93   7.05   
...             ...     ...    ...     ...     ...      ...      ...    ...   
260 312     20.0037  0.7000  100.0  491.19  608.79  1495.60  1269.51   9.35   
    313     10.0022  0.2510  100.0  489.05  605.81  1514.32  1324.12  10.52   
    314     25.0041  0.6200   60.0  462.54  537.48  1276.24  1057.92   7.05   
    315     25.0033  0.6220   60.0  462.54  537.84  1272.95  1066.30   7.05   
    316     35.0036  0.8400  100.0  449.44  556.64  1374.61  1145.52   5.48   

               10      11  ...      16       17       18       19    20   21  \
id  flight                 ...                                                 
1   1        8.00  194.64  ...  183.06  2387.72  8048.56   9.3461  0.02  334   
    2        5.71  138.51  ...  130.42  2387.66  8072.30   9.3774  0.02  330   
    3        9.02  175.71  ...  164.22  2028.03  7864.87  10.8941  0.02  309   
    4        5.71  138.46  ...  130.72  2387.61  8068.66   9.3528  0.02  329   
    5        9.03  175.05  ...  164.31  2028.00  7861.23  10.8963  0.02  309   
...           ...     ...  ...     ...      ...      ...      ...   ...  ...   
260 312     13.66  334.00  ...  314.05  2389.02  8169.64   9.3035  0.03  369   
    313     15.50  392.46  ...  371.22  2388.42  8245.36   8.7586  0.03  374   
    314      9.04  175.11  ...  163.74  2030.33  7971.25  11.0657  0.02  310   
    315      9.04  174.81  ...  164.37  2030.35  7972.47  11.0537  0.02  311   
    316      8.01  194.33  ...  183.09  2390.38  8185.35   9.3998  0.02  338   

              22      23     24       25  
id  flight                                
1   1       2223  100.00  14.73   8.8071  
    2       2212  100.00  10.41   6.2665  
    3       1915   84.93  14.08   8.6723  
    4       2212  100.00  10.59   6.4701  
    5       1915   84.93  14.13   8.5286  
...          ...     ...    ...      ...  
260 312     2324  100.00  24.36  14.5189  
    313     2319  100.00  28.10  16.9454  
    314     1915   84.93  14.19   8.5503  
    315     1915   84.93  14.05   8.3729  
    316     2223  100.00  14.75   8.8446  

[53759 rows x 24 columns]

In [2]:
jet.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53759 entries, (1, 1) to (260, 316)
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2       53759 non-null  float64
 1   3       53759 non-null  float64
 2   4       53759 non-null  float64
 3   5       53759 non-null  float64
 4   6       53759 non-null  float64
 5   7       53759 non-null  float64
 6   8       53759 non-null  float64
 7   9       53759 non-null  float64
 8   10      53759 non-null  float64
 9   11      53759 non-null  float64
 10  12      53759 non-null  float64
 11  13      53759 non-null  float64
 12  14      53759 non-null  float64
 13  15      53759 non-null  float64
 14  16      53759 non-null  float64
 15  17      53759 non-null  float64
 16  18      53759 non-null  float64
 17  19      53759 non-null  float64
 18  20      53759 non-null  float64
 19  21      53759 non-null  int64  
 20  22      53759 non-null  int64  
 21  23      53759 non-null  f

In [3]:
pd.isnull(jet).sum()

2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
dtype: int64

In [ ]:
'''
MMD functions implemented in tensorflow.
'''
from __future__ import division

import tensorflow as tf

from tf_ops import dot, sq_sum


_eps=1e-8

################################################################################
### Quadratic-time MMD with Gaussian RBF kernel

def _mix_rbf_kernel(X, Y, sigmas, wts=None):
    if wts is None:
        wts = [1] * len(sigmas)

    XX = tf.matmul(X, X, transpose_b=True)
    XY = tf.matmul(X, Y, transpose_b=True)
    YY = tf.matmul(Y, Y, transpose_b=True)

    X_sqnorms = tf.diag_part(XX)
    Y_sqnorms = tf.diag_part(YY)

    r = lambda x: tf.expand_dims(x, 0)
    c = lambda x: tf.expand_dims(x, 1)

    K_XX, K_XY, K_YY = 0, 0, 0
    for sigma, wt in zip(sigmas, wts):
        gamma = 1 / (2 * sigma**2)
        K_XX += wt * tf.exp(-gamma * (-2 * XX + c(X_sqnorms) + r(X_sqnorms)))
        K_XY += wt * tf.exp(-gamma * (-2 * XY + c(X_sqnorms) + r(Y_sqnorms)))
        K_YY += wt * tf.exp(-gamma * (-2 * YY + c(Y_sqnorms) + r(Y_sqnorms)))

    return K_XX, K_XY, K_YY, tf.reduce_sum(wts)


def rbf_mmd2(X, Y, sigma=1, biased=True):
    return mix_rbf_mmd2(X, Y, sigmas=[sigma], biased=biased)


def mix_rbf_mmd2(X, Y, sigmas=(1,), wts=None, biased=True):
    K_XX, K_XY, K_YY, d = _mix_rbf_kernel(X, Y, sigmas, wts)
    return _mmd2(K_XX, K_XY, K_YY, const_diagonal=d, biased=biased)


def rbf_mmd2_and_ratio(X, Y, sigma=1, biased=True):
    return mix_rbf_mmd2_and_ratio(X, Y, sigmas=[sigma], biased=biased)


def mix_rbf_mmd2_and_ratio(X, Y, sigmas=(1,), wts=None, biased=True):
    K_XX, K_XY, K_YY, d = _mix_rbf_kernel(X, Y, sigmas, wts)
    return _mmd2_and_ratio(K_XX, K_XY, K_YY, const_diagonal=d, biased=biased)


################################################################################
### Helper functions to compute variances based on kernel matrices


def _mmd2(K_XX, K_XY, K_YY, const_diagonal=False, biased=False):
    m = tf.cast(K_XX.get_shape()[0], tf.float32)
    n = tf.cast(K_YY.get_shape()[0], tf.float32)

    if biased:
        mmd2 = (tf.reduce_sum(K_XX) / (m * m)
              + tf.reduce_sum(K_YY) / (n * n)
              - 2 * tf.reduce_sum(K_XY) / (m * n))
    else:
        if const_diagonal is not False:
            trace_X = m * const_diagonal
            trace_Y = n * const_diagonal
        else:
            trace_X = tf.trace(K_XX)
            trace_Y = tf.trace(K_YY)

        mmd2 = ((tf.reduce_sum(K_XX) - trace_X) / (m * (m - 1))
              + (tf.reduce_sum(K_YY) - trace_Y) / (n * (n - 1))
              - 2 * tf.reduce_sum(K_XY) / (m * n))

    return mmd2


def _mmd2_and_ratio(K_XX, K_XY, K_YY, const_diagonal=False, biased=False,
                    min_var_est=_eps):
    mmd2, var_est = _mmd2_and_variance(
        K_XX, K_XY, K_YY, const_diagonal=const_diagonal, biased=biased)
    ratio = mmd2 / tf.sqrt(tf.maximum(var_est, min_var_est))
    return mmd2, ratio


def _mmd2_and_variance(K_XX, K_XY, K_YY, const_diagonal=False, biased=False):
    m = tf.cast(K_XX.get_shape()[0], tf.float32)  # Assumes X, Y are same shape

    ### Get the various sums of kernels that we'll use
    # Kts drop the diagonal, but we don't need to compute them explicitly
    if const_diagonal is not False:
        const_diagonal = tf.cast(const_diagonal, tf.float32)
        diag_X = diag_Y = const_diagonal
        sum_diag_X = sum_diag_Y = m * const_diagonal
        sum_diag2_X = sum_diag2_Y = m * const_diagonal**2
    else:
        diag_X = tf.diag_part(K_XX)
        diag_Y = tf.diag_part(K_YY)

        sum_diag_X = tf.reduce_sum(diag_X)
        sum_diag_Y = tf.reduce_sum(diag_Y)

        sum_diag2_X = sq_sum(diag_X)
        sum_diag2_Y = sq_sum(diag_Y)

    Kt_XX_sums = tf.reduce_sum(K_XX, 1) - diag_X
    Kt_YY_sums = tf.reduce_sum(K_YY, 1) - diag_Y
    K_XY_sums_0 = tf.reduce_sum(K_XY, 0)
    K_XY_sums_1 = tf.reduce_sum(K_XY, 1)

    Kt_XX_sum = tf.reduce_sum(Kt_XX_sums)
    Kt_YY_sum = tf.reduce_sum(Kt_YY_sums)
    K_XY_sum = tf.reduce_sum(K_XY_sums_0)

    Kt_XX_2_sum = sq_sum(K_XX) - sum_diag2_X
    Kt_YY_2_sum = sq_sum(K_YY) - sum_diag2_Y
    K_XY_2_sum  = sq_sum(K_XY)

    if biased:
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * m)
              + (Kt_YY_sum + sum_diag_Y) / (m * m)
              - 2 * K_XY_sum / (m * m))
    else:
        mmd2 = ((Kt_XX_sum + sum_diag_X) / (m * (m-1))
              + (Kt_YY_sum + sum_diag_Y) / (m * (m-1))
              - 2 * K_XY_sum / (m * m))

    var_est = (
          2 / (m**2 * (m-1)**2) * (
              2 * sq_sum(Kt_XX_sums) - Kt_XX_2_sum
            + 2 * sq_sum(Kt_YY_sums) - Kt_YY_2_sum)
        - (4*m-6) / (m**3 * (m-1)**3) * (Kt_XX_sum**2 + Kt_YY_sum**2)
        + 4*(m-2) / (m**3 * (m-1)**2) * (
              sq_sum(K_XY_sums_1) + sq_sum(K_XY_sums_0))
        - 4 * (m-3) / (m**3 * (m-1)**2) * K_XY_2_sum
        - (8*m - 12) / (m**5 * (m-1)) * K_XY_sum**2
        + 8 / (m**3 * (m-1)) * (
              1/m * (Kt_XX_sum + Kt_YY_sum) * K_XY_sum
            - dot(Kt_XX_sums, K_XY_sums_1)
            - dot(Kt_YY_sums, K_XY_sums_0))
    )

    return mmd2, var_est